<a href="https://colab.research.google.com/github/jamesstaub/Listen-Up/blob/main/audio_slicer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Dependencies & Import Libraries
!pip install -q umap-learn
!pip install -q librosa soundfile ipywidgets

import os
import glob
import numpy as np
import pandas as pd
import librosa
import soundfile as sf

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import umap
from scipy.spatial.distance import cdist


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00


In [5]:
from google.colab import drive
# @title 2. Mount Google Drive
drive.mount('/content/drive')
print("✅ Drive Mounted.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive Mounted.


In [1]:
CONFIG = {
    # ---- paths ----
    "input_folder": "/content/drive/My Drive/audio/sp-tools-corpora/preparedpiano",
    "output_folder": "/content/drive/My Drive/audio/sp-tools-corpora/preparedpiano/clusters",
    "data_file": "/content/drive/My Drive/audio/sp-tools-corpora/preparedpiano/clusters/analysis_data.csv",

    # ---- behavior ----
    "resume": True,        # load existing analysis if present
    "clear_existing": False,  # nuke data + clusters before run

    # ---- audio ----
    "sample_rate": 22050,
    "stereo_mode": "sum",
    "file_limit": 0,

    "min_duration": 0.1,
    "max_duration": 4.0,

    # ---- clustering ----
    "n_clusters": 8,

    # ---- filtering ----
    "similarity_threshold": 0.6, # 0.95 -> keeps most slices, 0.05 -> keep only the most unique slices

    "umap": {
        "n_neighbors": 15,
        "min_dist": 0.1
    },

}


In [2]:

def feature_columns(self, descriptor):
    if descriptor == "timbral":
        return [
            c for c in self.df.columns
            if c.startswith("mfcc_") or c.startswith("mfcc_delta_")
        ]
    elif descriptor == "tonal":
        return [
            c for c in self.df.columns
            if c.startswith("chroma_") or c.startswith("chroma_delta_")
        ]
    else:
        raise ValueError(f"Unknown descriptor: {descriptor}")

def stats(x):
    return {
        "mean": float(np.mean(x)),
        "min": float(np.min(x)),
        "max": float(np.max(x)),
        "std": float(np.std(x))
    }

def clear_existing_state(cfg):
    if os.path.exists(cfg["data_file"]):
        print("🧹 Removing existing analysis data")
        os.remove(cfg["data_file"])

    if os.path.exists(cfg["output_folder"]):
        print("🧹 Removing existing cluster folders")
        for root, dirs, files in os.walk(cfg["output_folder"], topdown=False):
            for f in files:
                os.remove(os.path.join(root, f))
            for d in dirs:
                os.rmdir(os.path.join(root, d))

In [11]:


class SampleBrain:
    def __init__(self, cfg):
        self.cfg = cfg
        self.sr = cfg["sample_rate"]
        self.df = pd.DataFrame()
        self.features = None

    def find_zero_crossing(self, y, idx):
        if idx <= 0 or idx >= len(y) - 1:
            return idx
        win = int(0.02 * self.sr)
        s = max(0, idx - win)
        e = min(len(y), idx + win)
        zc = np.where(np.diff(np.signbit(y[s:e])))[0]
        if len(zc):
            return s + zc[np.argmin(np.abs(zc - (idx - s)))]
        return idx

    def apply_envelope(self, y, fade_ms=5):
        n = int((fade_ms / 1000) * self.sr)
        if len(y) < 2 * n:
            return y
        env = np.ones(len(y))
        env[:n] = np.linspace(0, 1, n)
        env[-n:] = np.linspace(1, 0, n)
        return y * env

    def analyze(self):
        if self.cfg.get("resume") and os.path.exists(self.cfg["data_file"]):
            self.load_dataframe()
            print(f"✅ Loaded {len(self.df)} existing slices")
            return

        cfg = self.cfg

        # --- gather audio files ---
        files = []
        for ext in ("wav", "mp3", "aiff"):
            files += glob.glob(os.path.join(cfg["input_folder"], "**", f"*.{ext}"), recursive=True)
        if cfg.get("file_limit"):
            files = files[:cfg["file_limit"]]

        print(f"📂 Found {len(files)} audio files")
        rows = []

        # --- process files ---
        for i, path in enumerate(files):
            print(f"🔍 [{i+1}/{len(files)}] {os.path.basename(path)}")
            try:
                y, _ = librosa.load(path, sr=self.sr, mono=False)

                # handle channels
                channels = []
                if y.ndim == 1:
                    channels = [("mono", y)]
                else:
                    if cfg["stereo_mode"] == "sum":
                        channels = [("sum", librosa.to_mono(y))]
                    elif cfg["stereo_mode"] == "split":
                        channels = [("L", y[0]), ("R", y[1])]
                    elif cfg["stereo_mode"] == "L":
                        channels = [("L", y[0])]
                    elif cfg["stereo_mode"] == "R":
                        channels = [("R", y[1])]

                for ch, sig in channels:
                    onsets = librosa.onset.onset_detect(y=sig, sr=self.sr, units="samples")
                    if len(onsets) == 0:
                        onsets = [0]

                    for j in range(len(onsets)):
                        s = onsets[j]
                        e = onsets[j + 1] if j + 1 < len(onsets) else len(sig)
                        s = self.find_zero_crossing(sig, s)
                        e = self.find_zero_crossing(sig, e)

                        dur = (e - s) / self.sr
                        if not (cfg["min_duration"] <= dur <= cfg["max_duration"]):
                            continue

                        slice_y = self.apply_envelope(sig[s:e])
                        y_fix = librosa.util.fix_length(slice_y, size=10240)

                        row = {
                            "file_path": path,
                            "channel": ch,
                            "duration": dur,
                            "start_sample": int(s),
                            "end_sample": int(e),
                            "rolloff": 0.0,
                            "attack_slope": 0.0,
                            "loudness": float(np.mean(librosa.feature.rms(y=y_fix))),
                        }

                        # --- MFCC (mean only, 13 coeffs) ---
                        mfcc = librosa.feature.mfcc(y=y_fix, sr=self.sr, n_mfcc=13)
                        mfcc_d = librosa.feature.delta(mfcc)
                        for idx in range(13):
                            row[f"mfcc_mean_{idx}"] = float(np.mean(mfcc[idx]))
                            row[f"mfcc_delta_mean_{idx}"] = float(np.mean(mfcc_d[idx]))

                        # --- Spectral rolloff ---
                        try:
                            ro = librosa.feature.spectral_rolloff(y=y_fix, sr=self.sr)
                            row["rolloff"] = float(np.mean(ro)) if ro.size > 0 else 0.0
                        except Exception:
                            row["rolloff"] = 0.0

                        # --- Attack slope (first 50ms RMS) ---
                        attack_len = int(0.05 * self.sr)
                        attack = slice_y[:attack_len]
                        if len(attack) > 8:
                            rms_env = librosa.feature.rms(y=attack)[0]
                            row["attack_slope"] = float(np.polyfit(np.arange(len(rms_env)), rms_env, 1)[0])
                        else:
                            row["attack_slope"] = 0.0

                        rows.append(row)

            except Exception as e:
                print(f"⚠️ Error: {e}")

        self.df = pd.DataFrame(rows)
        self.save_dataframe()
        print(f"✨ Extracted {len(self.df)} slices")


    def cluster(self, n_clusters=8, feature_weights=None, umap_config=None):
        """
        Clusters slices based on core timbral features.
        feature_weights: dict of feature_name -> multiplier (after scaling)
        umap_config: dict with 'n_neighbors' and 'min_dist'
        """
        if self.df.empty:
            print("⚠️ No slices available for clustering")
            return

        print(f"🧠 Clustering {len(self.df)} slices...")

        # ---- 1️⃣ Select core features ----
        feature_cols = [f"mfcc_mean_{i}" for i in range(13)] + ["loudness", "rolloff"]
        for col in feature_cols:
            if col not in self.df.columns:
                print(f"⚠️ Missing {col}, filling with 0")
                self.df[col] = 0.0

        X = self.df[feature_cols].values.astype(np.float32)

        # ---- 2️⃣ Scale features ----
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # ---- 3️⃣ Apply weights if provided ----
        if feature_weights:
            weights = np.array([feature_weights.get(c, 1.0) for c in feature_cols], dtype=np.float32)
            X_scaled *= weights

        # ---- 4️⃣ UMAP embedding ----
        umap_cfg = umap_config or {"n_neighbors": 15, "min_dist": 0.1}
        reducer = umap.UMAP(n_neighbors=umap_cfg["n_neighbors"],
                            min_dist=umap_cfg["min_dist"],
                            metric='euclidean',
                            random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        self.df["umap_x"], self.df["umap_y"] = embedding[:, 0], embedding[:, 1]

        # ---- 5️⃣ KMeans clustering ----
        n_samples = len(self.df)
        if n_clusters > n_samples:
            n_clusters = n_samples
            print(f"ℹ️ Adjusted n_clusters to {n_samples} (number of slices)")

        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
        self.df["cluster"] = kmeans.fit_predict(X_scaled)

        # ---- 6️⃣ Cluster summary ----
        print("📊 Cluster distribution:")
        print(self.df.groupby("cluster").size())

      # ----------------
      # Thinning logic: remove very similar slices after clustering
      # ----------------

    def thin(self, similarity_threshold=0.95, feature_cols=None, max_refs=300):
        """
        Scalable greedy thinning using cosine similarity.
        No NxN matrices. Safe for very large clusters.

        similarity_threshold: float (0.9–0.99 typical)
        max_refs: max number of reference slices per cluster
                  (caps runtime + memory)
        """

        if "cluster" not in self.df.columns:
            raise RuntimeError("Run clustering before thinning")

        feature_cols = feature_cols or [c for c in self.df.columns if c.startswith("mfcc_")]
        print(f"🔍 Thinning with cosine similarity > {similarity_threshold}")

        kept_rows = []

        for cid, cdf in self.df.groupby("cluster"):
            print(f"  • Cluster {cid}: {len(cdf)} slices")

            X = cdf[feature_cols].values.astype(np.float32)
            X = StandardScaler().fit_transform(X)

            kept_idx = []
            ref_vectors = []

            for i, vec in enumerate(X):
                if not ref_vectors:
                    kept_idx.append(i)
                    ref_vectors.append(vec)
                    continue

                sims = cosine_similarity(
                    vec.reshape(1, -1),
                    np.array(ref_vectors)
                )[0]

                if np.max(sims) < similarity_threshold:
                    kept_idx.append(i)
                    ref_vectors.append(vec)

                    # cap reference size to avoid slowdown
                    if len(ref_vectors) > max_refs:
                        ref_vectors = ref_vectors[-max_refs:]

            kept = cdf.iloc[kept_idx]
            kept_rows.append(kept)

            print(f"    ✂️ {len(cdf)} → {len(kept)} kept")

        self.df = pd.concat(kept_rows).reset_index(drop=True)
        print(f"✅ Final slice count: {len(self.df)}")


    def save_dataframe(self):
        path = self.cfg["data_file"]
        print(f"💾 Saving analysis data to {path}")

        df = self.df.copy()
        # df["raw_audio"] = df["raw_audio"].apply(lambda x: x.tolist())
        df.to_csv(path, index=False)

    def load_dataframe(self):
        path = self.cfg["data_file"]
        print(f"📂 Loading analysis data from {path}")

        df = pd.read_csv(path)
        # df["raw_audio"] = df["raw_audio"].apply(lambda x: np.array(eval(x)))
        self.df = df


    def export(self):
        out = self.cfg["output_folder"]
        os.makedirs(out, exist_ok=True)

        print(f"💾 Writing slices to {out}")


        audio_cache = {}

        for _, row in self.df.iterrows():
            path = row["file_path"]

            if path not in audio_cache:
                y, _ = librosa.load(path, sr=self.sr, mono=True)
                audio_cache[path] = y
            else:
                y = audio_cache[path]

            slice_y = y[row["start_sample"]:row["end_sample"]]
            # Write to cluster folder
            c_dir = os.path.join(self.cfg["output_folder"], f"Cluster_{row['cluster']}")
            os.makedirs(c_dir, exist_ok=True)
            fn = f"s_{_}_{row['channel']}.wav"
            sf.write(os.path.join(c_dir, fn), slice_y, self.sr)


        print("✅ Export complete")



In [21]:
if __name__ == "__main__":
    if CONFIG["clear_existing"]:
        clear_existing_state(CONFIG)

    brain = SampleBrain(CONFIG)

    brain.analyze()   # load or compute
    brain.cluster()   # fast, repeatable


📂 Loading analysis data from /content/drive/My Drive/audio/sp-tools-corpora/preparedpiano/clusters/analysis_data.csv
✅ Loaded 91570 existing slices
🧠 Clustering 91570 slices...


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


📊 Cluster distribution:
cluster
0    31608
1    10491
2     8728
3     8433
4     8477
5     9712
6     6712
7     7409
dtype: int64


In [22]:
brain.thin(.5)

🔍 Thinning with cosine similarity > 0.3
  • Cluster 0: 31608 slices
    ✂️ 31608 → 40 kept
  • Cluster 1: 10491 slices
    ✂️ 10491 → 39 kept
  • Cluster 2: 8728 slices
    ✂️ 8728 → 49 kept
  • Cluster 3: 8433 slices
    ✂️ 8433 → 45 kept
  • Cluster 4: 8477 slices
    ✂️ 8477 → 47 kept
  • Cluster 5: 9712 slices
    ✂️ 9712 → 51 kept
  • Cluster 6: 6712 slices
    ✂️ 6712 → 36 kept
  • Cluster 7: 7409 slices
    ✂️ 7409 → 27 kept
✅ Final slice count: 334


In [23]:
brain.export()

💾 Writing slices to /content/drive/My Drive/audio/sp-tools-corpora/preparedpiano/clusters
✅ Export complete
